In [33]:
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
!pip install bert-tensorflow
!pip install transformers
# %load_ext tensorboard  //use it for tensorboard
import datetime

from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# rm -rf ./logs/   #use it for tensorboard

In [34]:
train_data = pd.read_csv('/content/drive/MyDrive/winter_project/train_data.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/winter_project/valid_data.csv')

train_data = train_data[['transcription', 'action', 'object', 'location']]
valid_data = valid_data[['transcription', 'action', 'object', 'location']]

train_data = train_data.dropna()
train_data["transcription"] = train_data["transcription"].str.lower()
valid_data = valid_data.dropna()
valid_data["transcription"] = valid_data["transcription"].str.lower()

train_data['action_label'] = pd.Categorical(train_data['action'])
train_data['object_label'] = pd.Categorical(train_data['object'])
train_data['location_label'] = pd.Categorical(train_data['location'])

valid_data['action_label'] = pd.Categorical(valid_data['action'])
valid_data['object_label'] = pd.Categorical(valid_data['object'])
valid_data['location_label'] = pd.Categorical(valid_data['location'])


train_data['action'] = train_data['action_label'].cat.codes
train_data['object'] = train_data['object_label'].cat.codes
train_data['location'] = train_data['location_label'].cat.codes

valid_data['action'] = valid_data['action_label'].cat.codes
valid_data['object'] = valid_data['object_label'].cat.codes
valid_data['location'] = valid_data['location_label'].cat.codes

action_dict = dict(zip(train_data.action, train_data.action_label))
object_dict = dict(zip(train_data.object, train_data.object_label))
location_dict = dict(zip(train_data.location, train_data.location_label))

In [35]:
print(train_data.shape)
print(valid_data.shape)
display(valid_data.head())

(11566, 7)
(3118, 7)


,transcription,action,object,location,action_label,object_label,location_label
0,turn on the lights,0,7,2,activate,lights,none
1,turn off the lights,3,7,2,deactivate,lights,none
2,change language,2,10,2,change language,none,none
3,pause the music,3,8,2,deactivate,music,none
4,resume,0,8,2,activate,music,none


In [36]:
max_length = 13
model_name = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [38]:
bert = transformer_model.layers[0]
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

Action = Dense(units=len(train_data.action_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='action')(pooled_output)
Object = Dense(units=len(train_data.object_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='object')(pooled_output)
Location = Dense(units=len(train_data.location_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='location')(pooled_output)

outputs = {'action': Action, 'object': Object, 'location' : Location}
model = Model(inputs = inputs, outputs = outputs, name='BERT_MultiLabel_MultiClass')

print(model.summary())

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 13)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
__________________________________________________________________________________________________
pooled_output (Dropout)         (None, 768)          0           bert[1][1]                       
__________________________________________________________________________________________________
action (Dense)                  (None, 6)            4614        pooled_output[0][0]              
_________________________________________________________________________

In [ ]:
# use it for tensorboard
# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)     

optimizer = Adam(learning_rate = 5e-05, epsilon = 1e-08, decay = 0.01, clipnorm = 1.0)

loss = {'action': CategoricalCrossentropy(from_logits = True), 'object': CategoricalCrossentropy(from_logits = True), 'location' : CategoricalCrossentropy(from_logits = True)}
metric = {'action': CategoricalCrossentropy('accuracy'), 'object': CategoricalCrossentropy('accuracy'), 'location' : CategoricalCrossentropy('accuracy')}

model.compile(optimizer = optimizer, loss = loss, metrics = metric)

y_train_action = to_categorical(train_data['action'])
y_train_object = to_categorical(train_data['object'])
y_train_location = to_categorical(train_data['location'])

X_train = tokenizer(text = train_data['transcription'].to_list(), add_special_tokens = True, max_length = max_length, truncation = True, padding=True, return_tensors='tf', return_token_type_ids = False, return_attention_mask = False, verbose = True)

history = model.fit(x = {'input_ids': X_train['input_ids']}, y = {'action': y_train_action, 'object': y_train_object, 'location' : y_train_location}, validation_split = 0.2, batch_size = 128, epochs = 5) 
                    #callbacks=[tensorboard_callback]) #use for tensorboard

In [ ]:
# %tensorboard --logdir logs/fit

In [22]:
valid_y_action = to_categorical(valid_data['action'])
valid_y_object = to_categorical(valid_data['object'])
valid_y_location = to_categorical(valid_data['location'])
valid_x = tokenizer(text = valid_data['transcription'].to_list(), add_special_tokens = True, max_length = max_length, truncation = True, padding = True,  return_tensors = 'tf',
                    return_token_type_ids = False, return_attention_mask = False, verbose = True)
valid_pred = model.predict(x = {'input_ids': valid_x['input_ids']})

In [ ]:
pred_action_id = np.argmax(np.array(valid_pred['action']), axis = 1)
pred_object_id = np.argmax(np.array(valid_pred['object']), axis = 1)
pred_location_id = np.argmax(np.array(valid_pred['location']), axis = 1)


pred_y_action_id = np.argmax(np.array(valid_y_action), axis = 1)
pred_y_object_id = np.argmax(np.array(valid_y_object), axis = 1)
pred_y_location_id = np.argmax(np.array(valid_y_location), axis = 1)

# np.unique(np.sum(np.stack([pred_action_id, pred_object_id, pred_location_id], axis=1) == np.stack([pred_y_action_id, pred_y_object_id, pred_y_location_id], axis=1), axis = 1), return_counts=True)


In [24]:
from sklearn.metrics import f1_score
y_true = np.stack([pred_y_action_id, pred_y_object_id, pred_y_location_id], axis = 1)
y_pred = np.stack([pred_action_id, pred_object_id, pred_location_id], axis = 1)
f1_action = f1_score(pred_y_action_id, pred_action_id, average='micro')
f1_object = f1_score(pred_y_object_id, pred_object_id, average='micro')
f1_location = f1_score(pred_y_action_id, pred_action_id, average='micro')
f1_total = (f1_action + f1_object + f1_location)/3
print(f1_total)

1.0

In [28]:
valid_data["action_pred_int"] = pred_y_action_id
valid_data["object_pred_int"] = pred_y_object_id
valid_data["location_pred_int"] = pred_y_location_id

valid_data["action_pred"] = valid_data["action"].map(lambda x : action_dict[x])
valid_data["object_pred"] = valid_data["object"].map(lambda x : object_dict[x])
valid_data["location_pred"] = valid_data["location"].map(lambda x : location_dict[x])


In [ ]:
valid_data = valid_data.drop(columns=["action_pred_int", "object_pred_int", "location_pred_int", "action", "location", "object"])
display(valid_data)

In [ ]:

############test evaluation############
test_y_action = to_categorical(test_data['action'])
test_y_object = to_categorical(test_data['object'])
test_y_location = to_categorical(test_data['location'])
test_x = tokenizer(text = test_data['transcription'].to_list(), add_special_tokens = True, max_length = max_length, truncation = True, padding = True,  return_tensors = 'tf',
                    return_token_type_ids = False, return_attention_mask = False, verbose = True)
test_pred = model.predict(x = {'input_ids': test_x['input_ids']})

pred_action_id = np.argmax(np.array(test_pred['action']), axis = 1)
pred_object_id = np.argmax(np.array(test_pred['object']), axis = 1)
pred_location_id = np.argmax(np.array(test_pred['location']), axis = 1)


pred_y_action_id = np.argmax(np.array(test_y_action), axis = 1)
pred_y_object_id = np.argmax(np.array(test_y_object), axis = 1)
pred_y_location_id = np.argmax(np.array(test_y_location), axis = 1)

# np.unique(np.sum(np.stack([pred_action_id, pred_object_id, pred_location_id], axis=1) == np.stack([pred_y_action_id, pred_y_object_id, pred_y_location_id], axis=1), axis = 1), return_counts=True)


y_true = np.stack([pred_y_action_id, pred_y_object_id, pred_y_location_id], axis = 1)
y_pred = np.stack([pred_action_id, pred_object_id, pred_location_id], axis = 1)
f1_action = f1_score(pred_y_action_id, pred_action_id, average='micro')
f1_object = f1_score(pred_y_object_id, pred_object_id, average='micro')
f1_location = f1_score(pred_y_action_id, pred_action_id, average='micro')
f1_total = (f1_action + f1_object + f1_location)/3
print(f1_total)

test_data["action_pred_int"] = pred_y_action_id
test_data["object_pred_int"] = pred_y_object_id
test_data["location_pred_int"] = pred_y_location_id

test_data["action_pred"] = test_data["action"].map(lambda x : action_dict[x])
test_data["object_pred"] = test_data["object"].map(lambda x : object_dict[x])
test_data["location_pred"] = test_data["location"].map(lambda x : location_dict[x])

test_data = test_data.drop(columns=["action_pred_int", "object_pred_int", "location_pred_int", "action", "location", "object"])
display(test_data)